# 1. Create SQL database for traffic crash datasets

EDA Notes:
 * accident_year in traffic_crash_all is inaccurate. There are incidents in 2025 and 2026

In [17]:
#import libraries 
import sqlite3
import pandas as pd

In [16]:
#list the csv files
csv_files = [
    ('Data/Traffic_Crashes_Resulting_in_Fatality_20240819.csv', 'traffic_crash_fatality'),
    ('Data/Traffic_Crashes_Resulting_in_Injury__Parties_Involved_20240819.csv', 'traffic_crash_parties'),
    ('Data/Traffic_Crashes_Resulting_in_Injury__Victims_Involved_20240819.csv', 'traffic_crash_victims'),
    ('Data/Traffic_Crashes_Resulting_in_Injury_20240819.csv', 'traffic_crash_all')
]

#create/connect to sqlite database
conn = sqlite3.connect("traffic_crash.db")

#import the data into the correct table
for csv_file, table_name in csv_files:
    #read the file
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)

#close the connect
conn.close()

C:\Users\yong2\AppData\Local\Temp\ipykernel_26604\3554752354.py:15: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
C:\Users\yong2\AppData\Local\Temp\ipykernel_26604\3554752354.py:15: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
C:\Users\yong2\AppData\Local\Temp\ipykernel_26604\3554752354.py:15: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [ ]:
#connect to the database
conn = sqlite3.connect('traffic_crash.db')

# Create a cursor object
cursor = conn.cursor()



In [77]:
#get the total number of car incidents, number killed and injured for each year
!sqlite3 traffic_crash.db -header -column   \
"SELECT \
SUBSTR(collision_date, 1, 4) AS incident_year,  \
COUNT(*) AS incidents_per_year, \
SUM(number_killed) AS number_killed_per_year,   \
SUM(number_injured) AS number_injured_per_year  \
FROM traffic_crash_all  \
GROUP BY incident_year;"



incident_year  incidents_per_year  number_killed_per_year  number_injured_per_year
-------------  ------------------  ----------------------  -----------------------
2005           3257                27.0                    4437                   
2006           2896                30.0                    3806                   
2007           3065                42.0                    3957                   
2008           3039                27.0                    3864                   
2009           2900                31.0                    3702                   
2010           3095                22.0                    3942                   
2011           3097                28.0                    3842                   
2012           3270                29.0                    4018                   
2013           3071                35.0                    3854                   
2014           2967                31.0                    3675                   
2015

In [76]:
#get the top 10 streets with the most incidents
!sqlite3 traffic_crash.db -header -column   \
"SELECT \
primary_rd, \
COUNT (*) AS n_incidents    \
FROM traffic_crash_all  \
GROUP BY primary_rd \
ORDER BY n_incidents DESC   \
LIMIT 10;"



primary_rd     n_incidents
-------------  -----------
MISSION ST     2560       
MARKET ST      2129       
VAN NESS AVE   1452       
GEARY BLVD     1156       
03RD ST        1099       
19TH AVE       880        
FULTON ST      861        
ALEMANY BLVD   861        
CALIFORNIA ST  834        
FOLSOM ST      810        


In [78]:
# which road surface incidents (e.g. wet, dry) has the most incidents
!sqlite3 traffic_crash.db -header -column   \
"WITH road_surface_cte AS ( \
SELECT COUNT(*) as n_incidents, road_surface    \
from traffic_crash_all  \
GROUP BY road_surface)  \
SELECT road_surface \
FROM road_surface_cte   \
ORDER BY n_incidents DESC   \
LIMIT 1;"


road_surface
------------
Dry         


In [82]:
# when do collisions tend to happen?
!sqlite3 traffic_crash.db -header -column \
"select time_cat, COUNT(*) AS n_incidents   \
    from traffic_crash_all  \
    group by time_cat   \
    ORDER BY n_incidents DESC;"

time_cat             n_incidents
-------------------  -----------
2:01 pm to 6:00 pm   15874      
10:01 am to 2:00 pm  12197      
6:01 pm to 10:00 pm  12159      
6:01 am to 10:00 am  10092      
10:01 pm to 2:00 am  6377       
2:01 am to 6:00 am   2731       
                     52         


In [ ]:
# are incidents involving bikes more likely to have more deaths?
#neighboords with the most incidents
# type of collision 
# when do collisions tend to happen

In [62]:
!sqlite3 traffic_crash.db -header -column \
"select * from traffic_crash_all limit 10;"

unique_id  cnn_intrsctn_fkey  cnn_sgmt_fkey  case_id_pkey  tb_latitude       tb_longitude       geocode_source   geocode_location  collision_datetime      collision_date    collision_time  accident_year  month     day_of_week  time_cat             juris  officer_id  reporting_district  beat_number  primary_rd      secondary_rd    distance  direction   weather_1  weather_2   collision_severity          type_of_collision   mviw                 ped_action                   road_surface  road_cond_1           road_cond_2  lighting              control_device   intersection          vz_pcf_code        vz_pcf_group  vz_pcf_description  vz_pcf_link  number_killed  number_injured  street_view                                                                          dph_col_grp  dph_col_grp_description   party_at_fault  party1_type  party1_dir_of_travel  party1_move_pre_acc  party2_type  party2_dir_of_travel  party2_move_pre_acc  point                                data_as_of              data_

In [45]:
!sqlite3 traffic_crash.db -header -column \
"PRAGMA table_info(traffic_crash_all);"

cid  name                          type     notnull  dflt_value  pk
---  ----------------------------  -------  -------  ----------  --
0    unique_id                     INTEGER  0                    0 
1    cnn_intrsctn_fkey             REAL     0                    0 
2    cnn_sgmt_fkey                 REAL     0                    0 
3    case_id_pkey                  TEXT     0                    0 
4    tb_latitude                   REAL     0                    0 
5    tb_longitude                  REAL     0                    0 
6    geocode_source                TEXT     0                    0 
7    geocode_location              TEXT     0                    0 
8    collision_datetime            TEXT     0                    0 
9    collision_date                TEXT     0                    0 
10   collision_time                TEXT     0                    0 
11   accident_year                 INTEGER  0                    0 
12   month                         TEXT     0   